In [1]:
import neuraltda.topology2 as tp2
import pandas as pd
from joblib import Parallel, delayed
import neuraltda.simpComp as sc
import neuraltda.topology2 as tp2
import neuraltda.spectralAnalysis as sa
import datetime
import pickle
import glob
import os
from ephys import core, events, clust
import numpy as np
import tqdm
import matplotlib.pyplot as plt
import h5py as h5
%matplotlib inline
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')

/home/brad/.conda/envs/neuraltda-3/lib/python3.6/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
# Rebin with 5ms, no overlap, start+2500ms to end+300ms, correct trials
block_path = '/home/brad/emily/P01S15/'
block_path = '/home/brad/emily/P01S13/'
block_path = '/home/brad/emily/P01S18/'
winsize = 5.0
dtover = 0.0
segment = [2500, 300]

cluster_group = ['Good', 'MUA']
#widenarrow_threshold = 0.000230 # sw threshold in seconds P01S15
widenarrow_threshold = 0.000240 #P01S18

In [3]:
spikes = core.load_spikes(block_path)
trials = events.load_trials(block_path)
fs = core.load_fs(block_path)

# Get wide/narrow clusters
clusters = core.load_clusters(block_path)
clusters_list = clusters[clusters.quality.isin(cluster_group)]['cluster'].unique()
(wide, narrow) = clust.get_wide_narrow(block_path, clusters_list, widenarrow_threshold)

# Get Correct/Incorrect Trials
correctTrials = trials[trials['correct']==True]
incorrectTrials = trials[trials['correct']==False]
passiveTrials = trials[-(trials['correct'].isin([True, False]))]

print(trials['stimulus'].unique())


['Repeat_15_1052sec5_sqzdn_65db' 'Repeat_58_1052sec3_sqzup_65db'
 'Repeat_23_C22sec2_sqzdn_65db' 'Repeat_44_B32sec1_sqzup_65db' 'T58S58D-1'
 'T58S58D23' 'T58S44D23' 'T58S44D-1' 'T-1S-1D23' 'T58S44D21' 'T58S58D7'
 'T23S15D-1' 'T23S23D58' 'T23S23D-1' 'T-1S-1D58' 'T23S15D58' 'T23S23D52'
 'T23S15D60' 'T58S58D15' 'T58S44D15' 'T-1S-1D15' 'T23S15D44' 'T23S23D44'
 'T-1S-1D44']


In [7]:
incorrectTrials['stimulus'].unique()

array(['T58S44D-1', 'T58S58D23', 'T58S44D23', 'T58S58D7', 'T58S58D-1',
       'T58S44D21', 'T23S15D58', 'T23S15D-1', 'T23S23D58', 'T23S23D52',
       'T58S58D15', 'T58S44D15', 'T23S15D44', 'T23S23D44'], dtype=object)